I need a function that takes the validation yaml and makes a long ugly string to paste into the csv data model.

In [117]:
import yaml
import pandas as pd
import os

In [2]:
# Load the YAML file
file_path = './ID_attribute_validation_rules.yml'

with open(file_path, 'r') as file:
    yaml_data = yaml.safe_load(file)

In [111]:
# per attribute, construct component-based rules per manifest and format as continuous string with separators expected by schematic

intermediate_dict = {}
transformed_dict = {}
transformed_dict['attribute_name'] = []
transformed_dict['attribute_rules'] = []

for attribute in yaml_data['attributes']:
    #attribute_rules = yaml_data['attributes']
    attribute_name = attribute['attribute']
    intermediate_dict['manifest_rules'] = []
    for manifest in attribute['rules_applied_to_manifests']:
        manifest_name = manifest['manifest']
        rule_string = ' '.join(manifest['rules'])
        level = manifest['level']
        manifest_rules = f'#{manifest_name} {rule_string} {level}^^'
        intermediate_dict['manifest_rules'].append(manifest_rules)
    transformed_dict['attribute_name'].append(attribute_name)
    transformed_dict['attribute_rules'].append(''.join(intermediate_dict['manifest_rules']))
        

In [114]:
# format as dataframe
rules_df = pd.DataFrame.from_dict(transformed_dict)

In [118]:
def find_csv_files(directory):
    csv_files = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith(".csv"):
                csv_files.append(os.path.join(root, file))
    return csv_files

In [121]:
csv_files = find_csv_files('modules/')

In [135]:
attributes_with_val_rules = []
for file in csv_files:
    for attribute in rules_df['attribute_name']:
        if attribute in os.path.basename(file):
            attributes_with_val_rules.append(file)


In [137]:
ind_df = pd.read_csv('modules/individual/individualID.csv')

In [165]:
rules_df.loc[rules_df['attribute_name'] == 'individualID']['attribute_rules']

0    #IndividualHumanMetadataTemplate matchExactlyO...
Name: attribute_rules, dtype: object

In [163]:
ind_df.iloc[0]['Validation Rules'] = str(rules_df.loc[rules_df['attribute_name'] == 'individualID', 'attribute_rules'])

/var/folders/dk/0kvw_jgs0x36k4d9nls882hc0000gq/T/ipykernel_87310/1027455809.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ind_df.iloc[0]['Validation Rules'] = str(rules_df.loc[rules_df['attribute_name'] == 'individualID', 'attribute_rules'])


In [162]:
ind_df.loc['']

0    '#IndividualHumanMetadataTemplate matchExactly...
Name: Validation Rules, dtype: object

In [136]:
# paste the validation rule string into the Validation Rules column for the manifest column row in each attribute csv

for attribute in attributes_with_val_rules:

    att_df = pd.read_csv(attribute)
    print(att_df)

    Attribute                                        Description  \
0  specimenID  Identifying string linked to a particular samp...   

   Valid Values  DependsOn  Properties  Required          Parent  \
0           NaN        NaN         NaN     False  ManifestColumn   

   DependsOn Component                                             Source  \
0                  NaN  sage.annotations-experimentalData.specimenID-0...   

                                    Validation Rules columnType       module  
0  '#BiospecimenMetadataTemplate unique^^required...     string  biospecimen  
      Attribute                                        Description  \
0  individualID  Identifying string linked to the individual or...   

   Valid Values  DependsOn  Properties  Required          Parent  \
0           NaN        NaN         NaN     False  ManifestColumn   

   DependsOn Component                                             Source  \
0                  NaN  sage.annotations-experimentalData.